In [ ]:
import os
import re
import pandas as pd
import numpy
import numpy as np
import tensorflow as tf
from tensorflow import keras
from gensim.models.fasttext import FastText
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import neighbors
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import gc
import nltk
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")


from nltk.corpus import stopwords
import multiprocessing

cores = multiprocessing.cpu_count()
FastTextModel = "fasttext_model.sav"

allContent = ""
path1 = "../input/ps-dataset/powershell_benign_dataset/"
path2 = "../input/ps-dataset/malicious_pure"
#path3 = r"D:\Test Scripts"
labels = []
text = []
testText = []
#new_str = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
fName= []
newText = []
regex = r':\s*\n+'
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

def test_bowser(path):
        with open(path, encoding='utf-8') as f:
            t_content = f.read()
            t_content = t_content.lower()
            t_content = t_content.replace("(", "")
            t_content = t_content.replace(")", "")
            t_content = t_content.replace("[", "")
            t_content = t_content.replace("]", "")
            t_content = t_content.replace("$", "")
            t_content = t_content.replace("-", "")
            t_content = t_content.replace("=", "")
            t_content = t_content.replace("", "")
            t_content = t_content.replace("{", "")
            t_content = t_content.replace("}", "")
            t_content = t_content.replace(".", "")
            t_content = t_content.replace("_", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace(":", "")
            t_content = t_content.replace("|", "")
            t_content = t_content.replace("/", "")
            t_content = t_content.replace("@", "")
            t_content = t_content.replace("?", "")
            t_content = t_content.replace(",", "")
            t_content = t_content.replace(">", "")
            t_content = t_content.replace("<", "")
            t_content = t_content.replace(";", "")
            t_content = t_content.replace("\\", "")
            t_content = t_content.replace("+", "")
            t_content = t_content.replace("\"","")
            t_content = t_content.replace("\'","")
            t_content = t_content.replace("`", "")
            t_content = t_content.replace("*", "")
            t_content = " ".join(t_content.split())
            newText.append(t_content)
            return t_content

def bowser(path, label):
    for filename in os.listdir(path):
        labelName = "__label__" + label
        labels.append(labelName)
        filename = os.path.join(path, filename)
        with open(filename, encoding='utf-8') as f:
                    content = f.read()
                    content = content.lower()
                    content = content.replace("(", "")
                    content = content.replace(")", "")
                    content = content.replace("[", "")
                    content = content.replace("]", "")
                    content = content.replace("$", "")
                    content = content.replace("-", "")
                    content = content.replace("!","")
                    content = content.replace("=", "")
                    content = content.replace("", "")
                    content = content.replace("{", "")
                    content = content.replace("}", "")
                    content = content.replace(".", "")
                    content = content.replace("_", "")
                    content = content.replace("|", "")
                    content = content.replace(":", "")
                    content = content.replace("|", "")
                    content = content.replace("/", "")
                    content = content.replace("@", "")
                    content = content.replace("?", "")
                    content = content.replace(",", "")
                    content = content.replace(">", "")
                    content = content.replace("<", "")
                    content = content.replace(";", "")
                    content = content.replace("\\", "")
                    content = content.replace("+", "")
                    content = content.replace("\"", "")
                    content = content.replace("\'", "")
                    content = content.replace("`", "")
                    content = content.replace("*", "")
                    content = " ".join(content.split())
                    testText.append(content)


#test_bowser(path3)
bowser(path1,"Benign")
bowser(path2,"Malicious")
#print(testText[6])
numFiles = len(testText)
#res = [re.sub(r"(\w)([A-Z])", r"\1 \2", ele) for ele in testText]
#print(res[1] + "\n")

df_main = pd.DataFrame(testText, columns=["Powershell Code"])
df_main['Labels'] = labels
#df.to_csv('tester.csv')

train, test = train_test_split(df_main, test_size=0.3, random_state=42)
train_labels = train['Labels']
train_samples = train['Powershell Code']
test_labels = test['Labels']
test_samples = test['Powershell Code']
#train_size = len(train['Powershell Code'])
train.reset_index()
train.reindex(index=range(0,5962))
#print(train_samples)

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

gc.collect()


In [2]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

vectorizer.get_vocabulary()[:5]

gc.collect()


In [3]:
path_to_glove_file = "../input/glove6b/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

gc.collect()


In [4]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
gc.collect()


In [5]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)
gc.collect()


In [6]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(labels), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

gc.collect()


In [7]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(df):
    df = reduce_mem_usage(df)
    return df


In [8]:


#x_train = vectorizer(train_samples)
#y_train = vectorizer(train_labels)


x_test = vectorizer(np.array([[s] for s in test_samples])).numpy()
y_test = vectorizer(np.array([[s] for s in test_labels])).numpy()

print(len(x_train))
print(len(y_train))

print(len(x_test))
print(len(y_test))
#x_val = vectorizer(np.array([[s] for s in test_samples])).numpy()

#y_train = np.array(train_labels)
#y_val = np.array(test_samples)